# Prognositc (trajectory prediction) performance of baseline ML models

* **Objectives:** 
    1. Predict Trajectory classes based on previously trained models (only MMSE based trajectories) 


* **Timepoints:**
    1. baseline (bl)
    2. baseline + follow-up (var_tp: since second tp can be from varying interval)


* **Input modalities:**
    1. clinical features (CS)
    2. structural features (CT)    
    3. both (CS+CT)


* **Models:** (10 model instances (per fold) are saved for each input combination)
    1. Logistic regression (Lasso)
    2. SVM
    3. Random Forest
    4. ANN
    5. LSM
    

In [1]:
# Basic Imports
import numpy as np
import pandas as pd
from scipy import stats
import pickle
import re
import collections
#import tables as tb
from math import isnan
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import KFold

%matplotlib inline
#plt.rcParams['figure.figsize'] = (15, 10)
import warnings; warnings.simplefilter('ignore') #to ignore old sKF iterator warning not used in this notebook

/opt/quarantine/anaconda/5.0.1-python3/install/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Naming:
    1. tp_name in ['bl','var_tp']
    2. modality in ['CS','CT','CS_CT']
    3. model in ['RFC','SVC','LR_L1','ANN','LSN']
    
# Order of variables
    1. ['CS(bl,tp)','AGE','APOE4', CT(bl,tp)]
    2. CT ROI order is grabbed from an exp_setup file

In [128]:
# Data dirs
model_dir = '/data/chamal/projects/nikhil/data/Traj_prediction/trained_models/'
project_dir = '/data/chamal/projects/nikhil/data/ADB/'
adb_civet_dir = '{}/civet/'.format(project_dir)

#To grab order of CT ROIs
AAL_roi_names = '{}/AAL_ROI_names_number_map.txt'.format(project_dir)
sample_exp_setup_path = '{}/Exp_502_ALL_ADNI_traj_MMSE_tp_var_tp_sample_setup_grab_CT_order.pkl'.format(project_dir)


In [149]:
# Create ordered CT ROIs
exp_setup = pd.read_pickle(sample_exp_setup_path)
df = exp_setup['df']
ct_cols_bl = list(df.columns[pd.Series(df.columns).str.contains('CT_bl')])
ct_cols_tp = list(df.columns[pd.Series(df.columns).str.contains('CT_var_tp')])
#ct_cols

# roi - name map
aal_roi_names = pd.read_csv(AAL_roi_names,delimiter='\'',header=None)
aal_roi_names = aal_roi_names[[0,1,3]]
aal_roi_names.columns = ['id','roi','name']
aal_roi_names['roi'] = aal_roi_names['roi']+'_CT_bl'
aal_roi_names_dict = dict(zip((aal_roi_names['id'].astype(str)), aal_roi_names['roi']))
aal_roi_names_dict['0_x'] = 'background_L'
aal_roi_names_dict['0_y'] = 'background_R'

In [195]:
def get_trained_model(model_dir,tp,modality,model,fold):
    if model in ['RFC','SVC','LR_L1']:
        saved_model_path = '{}/reference_models/baseline_models_tp_{}_{}.pkl'.format(model_dir,tp,modality)
    elif model == 'ann': # need to modify this
        saved_model_path = '{}/ann_models/'.format(model_dir,tp,modality)
    elif model == 'lsn': # need to modify this
        saved_model_path = '{}/lsn_models/'.format(model_dir,tp,modality)
    else:
        print('unknown model')
    saved_model_data = pd.read_pickle(saved_model_path)
    
    # grab the scaler and classifier from the pickle
    if model in ['SVC']:
        scaler = saved_model_data[model]['scaler_list'][fold]
    else:
        scaler = []
    clf = saved_model_data[model]['parallel_result'][fold]['clf']
    #feat_imp = saved_model_data[model]['parallel_result'][fold]['feat_imp']
    #print('required input shape: {}'.format(feat_imp.shape))
    
    return scaler, clf

def read_AAL_summary(subject_dir,sub_idx):
    # Subject naming: ADB_0226_AAL_lobe_thickness_tlaplace_30mm_left.dat    
    subject_file = subject_dir + 'ADB_{}_AAL_lobe_thickness_tlaplace_30mm_{}.dat'
    sub_df = pd.DataFrame(columns=['ROI','value'])
    for hemi in ['left','right']:
        df = pd.read_csv(subject_file.format(sub_idx,hemi),header=1,delim_whitespace=True)    
        df = df[['#','Label']]
        df.rename(columns={'#':'ROI','Label':'value'},inplace=True)
        df = df[df['ROI']!='Total']        
        df = df[df['ROI']!='0']        
        sub_df = sub_df.append(df)
    
    sub_df_T = sub_df.set_index('ROI').T    
    sub_df_T['sub_idx'] = 'ADB{}'.format(sub_idx)
    return sub_df_T


# Read clinical data
- this is for entire cohort

In [218]:
# Read demo data
adb_demo = pd.read_csv('{}/csv/ADB_scanqc_2018-07-05_ST.csv'.format(project_dir))
adb_demo.rename(columns={'ID':'sub_idx'},inplace=True)
_,adb_demo['sub_number'] = adb_demo['sub_idx'].str.split('B', 1).str
adb_demo.dropna(inplace=True)

# Read apoe4 data
adb_apoe = pd.read_csv('{}/csv/adb_apoestatus.csv'.format(project_dir))
adb_apoe.rename(columns={'subject':'sub_idx'},inplace=True)
adb_apoe = adb_apoe[~adb_apoe['apoe_genotype'].isnull()]
adb_apoe['apoe_A'],adb_apoe['apoe_B'] = adb_apoe['apoe_genotype'].str.split('-', 1).str
adb_apoe['apoe_A4'] = 0
adb_apoe['apoe_B4'] = 0
adb_apoe.loc[adb_apoe['apoe_A']=='4','apoe_A4'] = 1
adb_apoe.loc[adb_apoe['apoe_B']=='4','apoe_B4'] = 1
adb_apoe['APOE_status'] = adb_apoe['apoe_A4'] + adb_apoe['apoe_B4']
adb_apoe.dropna(inplace=True)

# Read MMSE data
adb_mmse = pd.read_csv('{}/csv/ADB_MMSE_grant_2017-03-24.csv'.format(project_dir))
adb_mmse.rename(columns={'subject_ID':'sub_idx','total_man':'mmse_calc','total_world_man':'mmse_world'},inplace=True)
adb_mmse.dropna(inplace=True)

# Read civet data
- this is per subject

In [231]:
civet_df_concat = pd.DataFrame()

for sub_idx in adb_demo['sub_number'].values:
    try:
        civet_df = read_AAL_summary(adb_civet_dir,sub_idx)
        if len(civet_df_concat) == 0:
            civet_df_concat = civet_df
        else:
            civet_df_concat = civet_df_concat.append(civet_df)
    except:
        print('subject not found {}'.format(sub_idx))
    
civet_df_concat.rename(columns=aal_roi_names_dict,inplace=True)

# ***tmp*** include dummy IPL columns (civet 2.1 has made them obsolete)
civet_df_concat['IPL.L_CT_bl'] = 0
civet_df_concat['IPL.R_CT_bl'] = 0

# reorder based on trained model expectations
civet_df_concat = civet_df_concat[['sub_idx']+ct_cols_bl]

subject not found 0069
subject not found 0195
subject not found 0221
subject not found 0229


# Merge all input datatypes 

In [220]:
# Create X matrix (order: ['CS(bl,tp)','AGE','APOE4', CT(bl,tp)])
input_data = pd.merge(adb_demo,adb_mmse,on='sub_idx',how='inner')
input_data = pd.merge(input_data,adb_apoe[['sub_idx','APOE_status']],on='sub_idx',how='inner')
input_data = pd.merge(input_data,civet_df_concat,on='sub_idx',how='inner')

# TODO # check mmse column with Mallar
X = input_data[['mmse_world','age','APOE_status']+ct_cols_bl].values
X.shape

(80, 81)

# Apply model

In [229]:
tp = 'bl'
modality = 'CS_CT'
model = 'LR_L1'
fold = 0

scaler, clf = get_trained_model(model_dir,tp,modality,model,fold)

if model in ['SVC']: # LR_L1 didn't make a difference with a scaler
    X = scaler.transform(X) 
    
traj_pred = clf.predict(X)

pred_df = input_data[['mmse_world','mmse_calc','age','APOE_status','group']]
pred_df['traj_pred_{}_{}_{}_{}'.format(tp,modality,model,fold)] = traj_pred

In [230]:
pred_df

,mmse_world,mmse_calc,age,APOE_status,group,traj_pred_bl_CS_CT_LR_L1_0
0,24.0,29.0,61,0,HC,1
1,30.0,30.0,64,0,FAMHX,0
2,24.0,29.0,68,0,HC,1
3,25.0,23.0,65,0,HC,1
4,27.0,27.0,78,0,HC,1
5,29.0,26.0,78,1,HC,1
6,29.0,29.0,69,2,HC,1
7,29.0,28.0,65,1,HC,1
8,29.0,29.0,71,1,HC,1
9,26.0,24.0,77,1,HC,1
